In [15]:
%pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [17]:
import gradio as gr
import requests

# Azure OpenAI 설정
AZURE_OPENAI_ENDPOINT = 'https://1507-ai.openai.azure.com/'
AZURE_OPENAI_API_KEY = '22646ec32d23457ca3ed253eefc5342f'
DEPLOYMENT_NAME = '1507-gpt4o'

# Azure OpenAI API 호출 함수  
def chatgpt_response(prompt, history):  
    print(prompt, history)
    headers = {  
        'Content-Type': 'application/json',  
        'api-key': AZURE_OPENAI_API_KEY  
    }  
    
    # Payload for the request
 
    messages = []
 
    #System
    messages.append(
        
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": "너는 서울 구로구의 쉼터에 대한 전문가야. 유저가 작성한 장소에서 어느 곳이 가까운지, 또 쉼터에 대한 정보를 알려줘"
            }
        ]
        },
    )
 
    #assistant
    if len(history) > 0:
        for text in history[0]:
            messages.append({
                "role": "assistant",
                "content": [
                {
                "type": "text",
                "text": text
                }
            ]
            })
 
    #user
    messages.append({
            "role": "user",
            "content": [
            {
            "type": "text",
            "text": prompt
            }
        ]
        })
 
    payload = {
    "messages": messages,
    "temperature": 0.7,
    "top_p": 0.95,
    "max_tokens": 4096
    }
  
    response = requests.post(  
        f"{AZURE_OPENAI_ENDPOINT}/openai/deployments/{DEPLOYMENT_NAME}/chat/completions?api-version=2024-02-15-preview",  
        headers=headers,  
        json=payload  
    )  
    
    result = response.json()  
    bot_response = result['choices'][0]['message']['content'].strip()
    history.append((prompt, bot_response))  
    return '', history

def greet(name):
    name_len = len(name)
    return f"Hello {name}! / {name_len}자"

with gr.Blocks() as demo:
    gr.Markdown("<h1 align='center'>ChatGPT RestSpace Demo</h1>")
    chatbot = gr.Chatbot()
    with gr.Column():
        with gr.Row():
            user_input = gr.Textbox(show_label=False, placeholder="Type your message here...", scale=3)
            submit_btn = gr.Button("Send", scale=1)
            # 비율 조정 3:1 비율
        with gr.Row():
            clear_btn = gr.Button("Clear")

    # 버튼 클릭 시 이벤트 처리
    submit_btn.click(chatgpt_response, [user_input, chatbot], [user_input, chatbot])
    clear_btn.click(lambda: None, None, chatbot, queue=False)
    # 엔터 처리
    user_input.submit(chatgpt_response, [user_input,chatbot], [user_input,chatbot])

# 인터페이스 실행
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7874

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


구로에서 쉬기 좋은 장소를 알려줘 []
위치도 알려줘 [['구로에서 쉬기 좋은 장소를 알려줘', '구로구에서 쉬기 좋은 쉼터를 찾으시나요? 구로구에는 여러 공원과 휴식 공간이 있어요. 몇 가지 추천드릴게요.\n\n1. **안양천변 공원**: 안양천을 따라 조성된 이 공원은 산책로와 자전거 도로가 잘 정비되어 있으며, 자연을 즐기기 좋은 곳입니다. 가족 단위로 가기에도 좋고, 운동을 하거나 산책을 즐기기에 적합합니다.\n\n2. **구로디지털단지**: 이 지역에는 IT 기업들이 많아 업무 공간이 주를 이루지만, 곳곳에 작은 공원과 쉼터가 있습니다. 특히 구로디지털단지역 주변에는 다양한 카페와 레스토랑도 있어 잠시 쉬어가기 좋습니다.\n\n3. **고척근린공원**: 고척동에 위치한 이 공원은 넓은 녹지와 운동 시설이 잘 갖추어져 있습니다. 아이들이 놀 수 있는 놀이시설도 마련되어 있어 가족 단위로 방문하기 좋습니다.\n\n4. **구로중앙공원**: 구로구청 근처에 위치한 이 공원은 산책로와 쉼터, 그리고 다양한 식물들이 있어 도심 속에서 자연을 느끼며 쉴 수 있는 공간입니다.\n\n5. **온수공원**: 온수역 인근에 위치한 공원으로, 조용한 분위기에서 산책하거나 벤치에 앉아 책을 읽기 좋은 곳입니다.\n\n이 외에도 구로구에는 작은 쉼터와 공원이 많이 있으니, 방문해 보시면 좋을 것 같습니다. 필요한 추가 정보가 있다면 언제든지 말씀해 주세요!']]
